In [40]:
import pandas as pd

In [41]:
df = pd.read_csv("spotify_millsongdata.csv")

In [42]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [43]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [44]:
df.shape

(57650, 4)

In [45]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [46]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [47]:
df.head(10)

,artist,song,text
0,Depeche Mode,Photographic,"A white house, a white room \r\nThe program o..."
1,Marilyn Manson,Choklit Factory,Choklit Factory \r\nThere was strange was a m...
2,Tina Turner,Tra La La La La,Walking along \r\n(Walking along) \r\nAnd I'...
3,Morrissey,Southpaw,You were a boy before you became a man \r\nI ...
4,Van Morrison,Help Me,You gotta help me \r\nCan't do it all by myse...
5,Nina Simone,Children Go Where I Send You,"Children go where I send you, \r\nHow shall I..."
6,Radiohead,Worry Wort,"You don't wanna talk to me, \r\nYou don't wan..."
7,The Jam,I GOT BY IN TIME,Saw a girl that I used to know \r\nI was deep...
8,Nazareth,May Heaven Keep You,"Beautiful, this love was meant to be \r\nInno..."
9,Tom T. Hall,Shame On The Rain,Shame on the rain for making me twice as blue ...


In [48]:
df['text'][0]

"A white house, a white room  \r\nThe program of today  \r\nLights on, switch on  \r\nYour eyes are far away  \r\nThe map represents you  \r\nAnd the tape is your voice  \r\nFollow all along you  \r\nTill you recognize the choice  \r\n  \r\nI take pictures  \r\nPhotographic pictures  \r\n  \r\nBright light  \r\nDark room  \r\nBright light  \r\nDark room  \r\n  \r\nI said I'd write a letter  \r\nBut I never got the time  \r\nAnd I'm looking to the day  \r\nI mesmerize the light  \r\n  \r\nThe years I spend just thinking  \r\nOf a moment we both knew  \r\nA second boss looking into  \r\nIt seems it can't be true\r\n\r\n"

In [49]:
# df = df.sample(5000)

In [50]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [51]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [52]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [53]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [56]:
similarity[0]

array([1.        , 0.022601  , 0.        , ..., 0.        , 0.00771796,
       0.04046698])

In [57]:
df[df['song'] == df['song'][1]]

,artist,song,text
1,Marilyn Manson,Choklit Factory,choklit factori there wa strang wa a man there...


In [58]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [59]:
recommendation(df['song'][1])

['Faster',
 'We Die Young',
 "There's The Girl",
 'Love And Memories',
 'A Vicious Kind',
 "Don't Blink",
 'Matrimony',
 'Stumbling In',
 'Master Of Puppets',
 "Everybody's On The Run",
 'Hey You',
 'Danger',
 'Dangerous',
 'Go Away',
 'Find Out',
 'Full On',
 'At My Window',
 "When We're Old Men",
 'All I Know',
 'All I Was Looking For Was You']

In [60]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))